In [2]:
# Se importan bibliotecas de interes
import pandas as pd
import numpy as np
import datetime
import calendar
import ast

# Procesamiento ETL de los datos

In [3]:
# Se carga el dataset "movies" en formato csv
df = pd.read_csv('..\Datasets\movies_dataset.csv', low_memory=False) 
'''se le agrega la condicion 'low_memory=False'para evitar la siguiente advertencia "DtypeWarning" 
'''
# Se revisa que se haya cargado correctamente
df.head(5)  
# Se cotejan los datos del dataset  
df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

'''
Con un total de 45466 filas (la primera corresponde a los 24 campos o nombres de las columnas),
en su mayoria conteniendo datos no especificados tipo 'objeto' (20) y 4 conteniendo datos de tipo 'float'.
De las 24 columnas, existen algunas con datos nulos < 50%:
'belongs_to_collection' (4494), 'homepage'(7782), 'tagline' (20412)
'''

In [4]:
# Se estima el numero de valores unicos del total del dataset
df.nunique()

adult                        5
belongs_to_collection     1698
budget                    1226
genres                    4069
homepage                  7673
id                       45436
imdb_id                  45417
original_language           92
original_title           43373
overview                 44307
popularity               43758
poster_path              45024
production_companies     22708
production_countries      2393
release_date             17336
revenue                   6863
runtime                    353
spoken_languages          1931
status                       6
tagline                  20283
title                    42277
video                        2
vote_average                92
vote_count                1820
dtype: int64

In [5]:
# Se estima el numero de duplicados del total del dataset
df.duplicated().sum()

17

### Criterios adoptados para ETL
En funcion de las consignas, se plantea un orden de pasos con el fin de economizar recursos, preservando la mayor cantidad de informacion necesaria pero priorizando aquella que sea considerada mas significativa para los fines del proyecto, intentando disminuir/minimizar las chances de errores/anomalias propias del procesamiento de datos:
* comenzando con la ejecucion de las consignas que promuevan eliminar campos o filas;
* siguiendo por aquellas consignas que requieran transformar los datos/campos preexistentes;
* por ultimo, aquellas consignas que requieran la generacion de nuevos campos o datos.

### Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage.

In [6]:
# Se eliminan las columnas que no serán utilizadas
columns_to_drop = ['video','imdb_id','adult',
                   'original_title','poster_path','homepage']
df.drop(columns_to_drop, axis=1, inplace=True) 
'''
El argumento axis=1 indica que afecta a columnas y el implace=True que afecta al DataFrame 
'''
# Se coteja nuevamente el estado del dataset 
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

In [7]:
# Se eliminan los duplicados en el campo 'id' para tratamiento posterior, 
# teniendo en cuenta de que se cargaron mas de una vez los mismos datos,
# consevando la primera fila para cada duplicado
df.drop_duplicates(subset=['id'], keep = 'first', inplace = True) 

# Se valida la ejecucion
df.duplicated().any()

False

### Los valores nulos del campo 'release date' deben eliminarse.

In [8]:
# Se contabilizan los valores nulos en el campo 'release_date'
print(df['release_date'].isnull().sum())

# Se eliminan las 87 filas con valores nulos en el campo 'release_date'

df.dropna(subset=['release_date'], inplace=True) 
'''
se coloca subset para que solo se eliminen las filas que contienen valores nulos en ese campo.
'''
# Se corrobora el estado del campo 'release date'
df['release_date'].info()


87
<class 'pandas.core.series.Series'>
Int64Index: 45349 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype 
--------------  ----- 
45349 non-null  object
dtypes: object(1)
memory usage: 708.6+ KB


De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [9]:
# Se corrobora el formato de la fecha 
df['release_date'].describe()

count          45349
unique         17336
top       2008-01-01
freq             136
Name: release_date, dtype: object

In [10]:
# Contar la cantidad de valores no nulos en la columna 'release_date'
print(df['release_date'].notnull().sum())
print(df['release_date'].isnull().sum())

45349
0


In [11]:
# Se convierte el campo 'release_date' a formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Se filtran filas con valores no válidos en 'release_date' y se imprimen
filas_no_validas = df[df['release_date'].isnull()]
print("Cantidad de filas no válidas:", len(filas_no_validas))


Cantidad de filas no válidas: 3


In [12]:
# Se eliminan las 3 filas que no tienen formato de fecha
df = df.drop(filas_no_validas.index)
print("Cantidad de filas no válidas eliminadas:", len(filas_no_validas))

Cantidad de filas no válidas eliminadas: 3


In [13]:
# Convertir la columna 'release_date' al formato de fecha deseado AAAA-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

# Extraer el año de la fecha de estreno y crear la columna 'release_year'
df['release_year'] = df['release_date'].dt.year.astype(int) 

# Extraer el mes de la columna "release_date" y crear la columna "release_month"
df['release_month'] = df['release_date'].dt.month.astype(int)
# Mapear el mes numérico al nombre del mes en español

meses_en_espanol = {
    1: 'enero',
    2: 'febrero',
    3: 'marzo',
    4: 'abril',
    5: 'mayo',
    6: 'junio',
    7: 'julio',
    8: 'agosto',
    9: 'septiembre',
    10: 'octubre',
    11: 'noviembre',
    12: 'diciembre'
}
df['release_month'] = df['release_month'].map(meses_en_espanol)

In [14]:
df.head(3)

belongs_to_collection    budget  \
0  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                NaN  65000000   
2  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   

                                              genres     id original_language  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862                en   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844                en   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602                en   

                                            overview popularity  \
0  Led by Woody, Andy's toys live happily in his ...  21.946943   
1  When siblings Judy and Peter discover an encha...  17.015539   
2  A family wedding reignites the ancient feud be...    11.7129   

                                production_companies  \
0     [{'name': 'Pixar Animation Studios', 'id': 3}]   
1  [{'name': 'TriStar Pictures', 'id': 559}, {'na...   
2  [{'name': 'Warner Bros.', 'id': 6194}, {'name'...   

                                production_countries release_date  \
0  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-10-30   
1  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-12-15   
2  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-12-22   

       revenue  runtime                                   spoken_languages  \
0  373554033.0     81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0    104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0    101.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   

              title  vote_average  vote_count  release_year release_month  
0         Toy Story           7.7      5415.0          1995       octubre  
1           Jumanji           6.9      2413.0          1995     diciembre  
2  Grumpier Old Men           6.5        92.0          1995     diciembre

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [15]:
# Rellenar los valores nulos en los campos "revenue" y "budget" con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [16]:
# Verificar tipo de dato (formato) para proceder al calculo propuesto
df['revenue'].info()
df['budget'].info()
# Convertir el formato del campo 'budget' a float
df['budget']= df['budget'].astype(float)
# Se verifica el nuevo formato
print(df['budget'].info())


<class 'pandas.core.series.Series'>
Int64Index: 45346 entries, 0 to 45465
Series name: revenue
Non-Null Count  Dtype  
--------------  -----  
45346 non-null  float64
dtypes: float64(1)
memory usage: 708.5 KB
<class 'pandas.core.series.Series'>
Int64Index: 45346 entries, 0 to 45465
Series name: budget
Non-Null Count  Dtype 
--------------  ----- 
45346 non-null  object
dtypes: object(1)
memory usage: 708.5+ KB
<class 'pandas.core.series.Series'>
Int64Index: 45346 entries, 0 to 45465
Series name: budget
Non-Null Count  Dtype  
--------------  -----  
45346 non-null  float64
dtypes: float64(1)
memory usage: 708.5 KB
None


In [17]:
# Se define función para calcular ROI de la pelicula
def retorno(celda):
    revenue = celda['revenue']
    budget = celda['budget']
    
    # Se establece si hay datos faltantes que el presupuesto es igual a cero
    try:
        if pd.isnull(revenue) or pd.isnull(budget) or budget == 0:
            return 0
    except ZeroDivisionError:
        return 0
        # Calcular 'return' de inversión a dos decimales
    return round(revenue / budget, 2)

# Se aplica la función a cada fila del dataset para crear la columna 'return'
df['return'] = df.apply(retorno, axis=1)

# Se corrobora su aplicacion
df['return']

0        12.45
1         4.04
2         0.00
3         5.09
4         0.00
         ...  
45460     0.00
45462     0.00
45463     0.00
45464     0.00
45465     0.00
Name: return, Length: 45346, dtype: float64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45465
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4485 non-null   object        
 1   budget                 45346 non-null  float64       
 2   genres                 45346 non-null  object        
 3   id                     45346 non-null  object        
 4   original_language      45335 non-null  object        
 5   overview               44405 non-null  object        
 6   popularity             45346 non-null  object        
 7   production_companies   45346 non-null  object        
 8   production_countries   45346 non-null  object        
 9   release_date           45346 non-null  datetime64[ns]
 10  revenue                45346 non-null  float64       
 11  runtime                45100 non-null  float64       
 12  spoken_languages       45346 non-null  object        
 13  s

### Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [19]:
# Se corrobora la informacion brindada en la consigna para cada campo mencionado
print(df['belongs_to_collection'].describe())
print(df['genres'].describe())
print(df['production_companies'].describe())
print(df['production_countries'].describe())
print(df['spoken_languages'].describe())

count                                                  4485
unique                                                 1695
top       {'id': 415931, 'name': 'The Bowery Boys', 'pos...
freq                                                     29
Name: belongs_to_collection, dtype: object
count                             45346
unique                             4065
top       [{'id': 18, 'name': 'Drama'}]
freq                               4994
Name: genres, dtype: object
count     45346
unique    22703
top          []
freq      11789
Name: production_companies, dtype: object
count                                                 45346
unique                                                 2389
top       [{'iso_3166_1': 'US', 'name': 'United States o...
freq                                                  17836
Name: production_countries, dtype: object
count                                        45346
unique                                        1931
top       [{'iso_639_1': 'en', 'name': 

Se procede a la ejecucion de la consigna integramente haciendo la observacion de que el campo "belongs_to_collection" posee una cantidad de datos que no alcanza el 10% de los datos totales del dataset (4494 filas no nulas de 45376 restantes) por lo que a mi criterio y a priori, careceria de relevancia para los fines del proyecto.

In [20]:
'''
Luego de una exhaustiva busqueda bibliografica y jornadas extensas de prueba y error,
se pudo afrontar esta consigna!
'''

# Se genera función auxiliar para desanidar los datos alojados en los campos requeridos
def desanidar(celda):
    try:
        return ast.literal_eval(celda)
    except (ValueError, SyntaxError):
        return np.nan

# Se desanida el campo 'belongs_to_collection'
df['belongs_to_collection'] = df['belongs_to_collection'].apply(desanidar)
df['collection_id'] = df['belongs_to_collection'].apply(lambda x: x['id'] if isinstance(x, dict) else np.nan)
df['collection_name'] = df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

# Se desanida el campo 'genres'
desanidar_genres = lambda x: ast.literal_eval(x) if pd.notnull(x) else []
df['genre_id'] = df['genres'].apply(lambda x: [g['id'] for g in desanidar_genres(x)])
df['genre_name'] = df['genres'].apply(lambda x: [g['name'] for g in desanidar_genres(x)])

# Se desanida el campo 'production_companies'
df['production_companies'] = df['production_companies'].apply(desanidar)
df['company_id'] = df['production_companies'].apply(lambda x: [c['id'] for c in x] if isinstance(x, list) else np.nan)
df['company_name'] = df['production_companies'].apply(lambda x: [c['name'] for c in x] if isinstance(x, list) else np.nan)

# Se desanida  el campo 'production_countries'
df['production_countries'] = df['production_countries'].apply(desanidar)
df['country_id'] = df['production_countries'].apply(lambda x: [c['iso_3166_1'] for c in x] if isinstance(x, list) else np.nan)
df['country_name'] = df['production_countries'].apply(lambda x: [c['name'] for c in x] if isinstance(x, list) else np.nan)

# Se desanida el campo 'spoken_languages'
df['spoken_languages'] = df['spoken_languages'].apply(desanidar)
df['language_id'] = df['spoken_languages'].apply(lambda x: [l['iso_639_1'] for l in x] if isinstance(x, list) else np.nan)
df['language_name'] = df['spoken_languages'].apply(lambda x: [l['name'] for l in x] if isinstance(x, list) else np.nan)

In [21]:
# Se corrobora la creacion de los nuevos campos desanidados y su inclusion en el DataFrame
df.head()
#df.info()

belongs_to_collection      budget  \
0  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000.0   
1                                                NaN  65000000.0   
2  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0.0   
3                                                NaN  16000000.0   
4  {'id': 96871, 'name': 'Father of the Bride Col...         0.0   

                                              genres     id original_language  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862                en   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844                en   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602                en   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357                en   
4                     [{'id': 35, 'name': 'Comedy'}]  11862                en   

                                            overview popularity  \
0  Led by Woody, Andy's toys live happily in his ...  21.946943   
1  When siblings Judy and Peter discover an encha...  17.015539   
2  A family wedding reignites the ancient feud be...    11.7129   
3  Cheated on, mistreated and stepped on, the wom...   3.859495   
4  Just when George Banks has recovered from his ...   8.387519   

                                production_companies  \
0     [{'name': 'Pixar Animation Studios', 'id': 3}]   
1  [{'name': 'TriStar Pictures', 'id': 559}, {'na...   
2  [{'name': 'Warner Bros.', 'id': 6194}, {'name'...   
3  [{'name': 'Twentieth Century Fox Film Corporat...   
4  [{'name': 'Sandollar Productions', 'id': 5842}...   

                                production_countries release_date  ...  \
0  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-10-30  ...   
1  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-12-15  ...   
2  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-12-22  ...   
3  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-12-22  ...   
4  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-02-10  ...   

   collection_id                 collection_name         genre_id  \
0        10194.0            Toy Story Collection  [16, 35, 10751]   
1            NaN                             NaN  [12, 14, 10751]   
2       119050.0       Grumpy Old Men Collection      [10749, 35]   
3            NaN                             NaN  [35, 18, 10749]   
4        96871.0  Father of the Bride Collection             [35]   

                     genre_name          company_id  \
0   [Animation, Comedy, Family]                 [3]   
1  [Adventure, Fantasy, Family]  [559, 2550, 10201]   
2             [Romance, Comedy]       [6194, 19464]   
3      [Comedy, Drama, Romance]               [306]   
4                      [Comedy]        [5842, 9195]   

                                        company_name  country_id  \
0                          [Pixar Animation Studios]        [US]   
1  [TriStar Pictures, Teitler Film, Interscope Co...        [US]   
2                     [Warner Bros., Lancaster Gate]        [US]   
3           [Twentieth Century Fox Film Corporation]        [US]   
4       [Sandollar Productions, Touchstone Pictures]        [US]   

                 country_name  language_id        language_name  
0  [United States of America]         [en]            [English]  
1  [United States of America]     [en, fr]  [English, Français]  
2  [United States of America]         [en]            [English]  
3  [United States of America]         [en]            [English]  
4  [United States of America]         [en]            [English]  

[5 rows x 31 columns]

In [22]:
# Se contabilizan los valores nulos en el campo 'collection_id'
print("Cantidad de registros nulos: " , df['collection_id'].isnull().sum())
print("Cantidad de registros no nulos: ", df['collection_id'].notnull().sum())

Cantidad de registros nulos:  40861
Cantidad de registros no nulos:  4485


In [23]:
# Se eliminan las columnas origen (desanidadas) que ya no serán utilizadas
columns_to_drop = ['belongs_to_collection','genres','production_companies',
                   'production_countries', 'spoken_languages']
df.drop(columns_to_drop, axis=1, inplace=True) 
'''
El argumento axis=1 indica que afecta a columnas y el implace=True que afecta al DataFrame 
'''
# Se coteja nuevamente el estado del dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45346 non-null  float64       
 1   id                 45346 non-null  object        
 2   original_language  45335 non-null  object        
 3   overview           44405 non-null  object        
 4   popularity         45346 non-null  object        
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45346 non-null  float64       
 7   runtime            45100 non-null  float64       
 8   status             45266 non-null  object        
 9   tagline            20387 non-null  object        
 10  title              45346 non-null  object        
 11  vote_average       45346 non-null  float64       
 12  vote_count         45346 non-null  float64       
 13  release_year       45346 non-null  int32         
 14  releas

In [24]:
# Se modifica el tipo de dato en el campo 'id' para poder unir posteriormente este dataset con el dataset de credits
df['id'] = pd.to_numeric(df['id'], errors='coerce')
'''
El parámetro errors='coerce' hace que valores inconsistentes o no numéricos
se reemplacen por NaN. 
'''
# Se eliminan filas con datos nulos en el campo 'id'
#df = df.dropna(subset=['id'])
df['id'].info()

<class 'pandas.core.series.Series'>
Int64Index: 45346 entries, 0 to 45465
Series name: id
Non-Null Count  Dtype
--------------  -----
45346 non-null  int64
dtypes: int64(1)
memory usage: 708.5 KB


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45346 non-null  float64       
 1   id                 45346 non-null  int64         
 2   original_language  45335 non-null  object        
 3   overview           44405 non-null  object        
 4   popularity         45346 non-null  object        
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45346 non-null  float64       
 7   runtime            45100 non-null  float64       
 8   status             45266 non-null  object        
 9   tagline            20387 non-null  object        
 10  title              45346 non-null  object        
 11  vote_average       45346 non-null  float64       
 12  vote_count         45346 non-null  float64       
 13  release_year       45346 non-null  int32         
 14  releas

In [26]:
df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'release_year', 'release_month', 'return',
       'collection_id', 'collection_name', 'genre_id', 'genre_name',
       'company_id', 'company_name', 'country_id', 'country_name',
       'language_id', 'language_name'],
      dtype='object')>

In [27]:
# Con la finalidad de economizar recursos y debido a la limitacion de 
# capacidad en pasos posteriores, se procede a una limpieza extra del dataset

# Se eliminan todas las filas cuyo 'revenue' , 'budget' y 'return' sea cero
df1 = df.drop(df[(df['revenue'] == 0) | (df['budget'] == 0) | (df['return'] == 0)].index)
df1.shape

(5313, 26)

In [28]:
# Se seleccionan los campos mas relevantes para las funciones requeridas de la API
columns_to_export = ['id', 'title', 'release_date', 'release_year', 'release_month',
                     'budget', 'revenue', 'return',
                     'popularity', 'vote_count','vote_average'
                     ]
 
# Se exporta csv para API
df_API = df1[columns_to_export]
df_API.to_csv('movies_ETL_API.csv', index=False)

In [29]:
df_API.shape

(5313, 11)

In [30]:
# Se organizan los campos para el metadataset
columns_order = ['id', 'title', 'release_date', 'release_year', 'release_month',
                     'budget', 'revenue', 'return',
                     'popularity', 'vote_count','vote_average',
                     'original_language', 'language_id', 'language_name',
                     'runtime', 'genre_id', 'genre_name',
                     'company_id', 'company_name', 'country_id', 'country_name'
                     ]
# Los siguientes campos se consideran prescindibles para EDA y MLOps
columns_ignored = ['status', 'overview', 'tagline', 'collection_id', 'collection_name'] 

# Se exporta csv metadataset
df_meta = df[columns_order]          
df_meta.to_csv('metadata_movies_ETL.csv', index=False)

In [31]:
df_meta.shape

(45346, 21)

In [38]:
# Se cargan datasets movies y credits para API
df1 = pd.read_csv('..\Datasets\movies_ETL_API.csv')
df2 = pd.read_csv('..\Datasets\credits_ETL.csv')

# Verificar el tipo de dato del campo 'id' en df1 y df2
print(df1['id'].dtype)
print(df2['id'].dtype)

# Se genera merge de datasets movies y credits para API
df_API = pd.merge(df1, df2, on='id', how='left')

# Se eliminan datos faltantes
df_API.dropna(axis=0 , inplace=True)

# Se exporta dataset para API
df_API.to_csv('movies_API.csv', index=False)

int64
int64


In [39]:
# Se convierte csv a zip para poder montar API
import zipfile

def compress_csv_to_zip(csv_filename, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(csv_filename)

# Se especifica el nombre del archivo CSV que deseas comprimir
csv_filename = 'movies_API.csv'

# Se especifica el nombre del archivo ZIP resultante
zip_filename = 'movies_API.zip'

# Se comprime el archivo CSV en ZIP
compress_csv_to_zip(csv_filename, zip_filename)

In [34]:
df_API.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5315 entries, 0 to 5318
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5315 non-null   int64  
 1   title          5315 non-null   object 
 2   release_date   5315 non-null   object 
 3   release_year   5315 non-null   int64  
 4   release_month  5315 non-null   object 
 5   budget         5315 non-null   float64
 6   revenue        5315 non-null   float64
 7   return         5315 non-null   float64
 8   popularity     5315 non-null   float64
 9   vote_count     5315 non-null   float64
 10  vote_average   5315 non-null   float64
 11  cast_name      5315 non-null   object 
 12  director_name  5315 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 581.3+ KB


In [40]:
# Se cargan datasets movies y credits para metadateset
df3 = pd.read_csv('..\Datasets\metadata_movies_ETL.csv')
df4 = pd.read_csv('..\Datasets\credits_ETL.csv')

# Verificar el tipo de dato del campo 'id' en df1 y df2
print(df3['id'].dtype)
print(df4['id'].dtype)

# Se genera merge de datasets movies y credits para MLOPs
df_metadata = pd.merge(df3, df4, on='id', how='left')

# Se exporta dataset para MLOps
df_metadata.to_csv('movies_metadata.csv', index=False)


int64
int64


In [41]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45390 entries, 0 to 45389
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45390 non-null  int64  
 1   title              45390 non-null  object 
 2   release_date       45390 non-null  object 
 3   release_year       45390 non-null  int64  
 4   release_month      45390 non-null  object 
 5   budget             45390 non-null  float64
 6   revenue            45390 non-null  float64
 7   return             45390 non-null  float64
 8   popularity         45390 non-null  float64
 9   vote_count         45390 non-null  float64
 10  vote_average       45390 non-null  float64
 11  original_language  45379 non-null  object 
 12  language_id        45390 non-null  object 
 13  language_name      45390 non-null  object 
 14  runtime            45144 non-null  float64
 15  genre_id           45390 non-null  object 
 16  genre_name         453